<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-PyTorch/blob/master/Chapter02/Sequential_method_to_build_a_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 使用Sequential方法构建神经网络

In [1]:
# 1. 定义小数据集
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [2]:
# 2. 导入相关的包并定义工作设备
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# 3. 定义数据集类
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    def __len__(self): 
        return len(self.x)

In [4]:
# 4. 定义数据集和数据加载器对象
ds = MyDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [5]:
# 5. 使用nn包中可用的Seuqnetial方法定义模型架构
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

In [6]:
# 6. 输出步骤5中所定义模型的摘要:

# 安装并导入使我们能够输出模型摘要的包
!pip install torch_summary
from torchsummary import summary

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [7]:
# 输出一个模型的摘要, 它需要输入模型的名称和模型的大小:
summary(model, torch.zeros(1,2));

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [8]:
# 7. 定义损失函数并训练模型
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(), lr = 0.001)
import time
loss_history = []
start = time.time()
for _ in range(50):
    for ix, iy in dl:
        opt.zero_grad()
        loss_value = loss_func(model(ix),iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.12802577018737793


In [9]:
# 8. 已经对模型进行了训练, 计算模型验证数据集上的预测值:

# 定义验证数据集:
val = [[8,9],[10,11],[1.5,2.5]]
# 转换成浮点tensor
val = torch.tensor(val).float()

In [10]:
# 前向过程
model(val.to(device))

tensor([[16.8192],
        [20.6739],
        [ 4.2186]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [11]:
# 上述代码的输出接近于预期的结果(输入值的总和)
val.sum(-1)


tensor([17., 21.,  4.])